In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
import numpy as np
import os
import torch.nn.functional as F


transform = T.Compose([
#     T.Resize((227,227)), 
    T.ToTensor(),  # 将Image转成Tensor，归一化至 [0.0, 1.0]
#     T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

class LoadDate(Dataset):
    
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = os.listdir(root)
        self.transforms = transform   
        
    def make_label(self, label):
        source = [str(i) for i in range(10)]
        source += [chr(i) for i in range(97, 97+26)]
        source += [chr(i) for i in range(65, 65+26)]
        string = ''.join(source)
        
        target = []
        for char in label:
            vec = [0] * 62
            vec[string.find(char)] = 1
            target += vec
        return torch.Tensor(target)
        
    def __getitem__(self, index):
        image_path = self.paths[index]    
        label = list(image_path)[:-4]
        label = self.make_label(label)

        pil_image = Image.open(self.root+image_path)
        if self.transforms:
            data = self.transforms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data, label

    def __len__(self):
        return len(self.paths)

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=310):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
#         self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(4608, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
#         out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(ResidualBlock)

In [3]:
data = LoadDate('train/train/', transform)
dataloader = DataLoader(data, batch_size=32, shuffle=True, drop_last=False)

In [5]:
cnn = ResNet18()
# loss_fn = nn.CrossEntropyLoss()

cnn.cuda()
# cnn.load_state_dict(torch.load('can2.pt'))

# cnn.eval()
loss_fn = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(cnn.parameters())


for i in range(7):  
    for j,(img,labels) in enumerate(dataloader):
        img = img.cuda()
        labels = labels.cuda()
        out = cnn(img)
        loss = loss_fn(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if j % 100 == 0:
            print('i=%d j=%d Loss: %.5f' %(i,j,loss.item()))

i=0 j=0 Loss: 0.72869
i=0 j=100 Loss: 0.11742


KeyboardInterrupt: 

In [9]:
torch.save(cnn.state_dict(),'can.pt')

In [ ]:
class LoadTest(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = os.listdir(root)
        self.transforms = transform   
        
    def __getitem__(self, index):
        image_path = self.paths[index]
        label = image_path
        if label !='.DS_Store':
            label = list(label)[:-4]
            label = int(''.join(label))
            pil_image = Image.open(self.root+image_path)   
            if self.transforms:
                data = self.transforms(pil_image)
            else:
                image_array = np.asarray(pil_image)
                data = torch.from_numpy(image_array)
            return data, label

    def __len__(self):
        return len(self.paths)

In [ ]:
def relabel(label):
    source = [str(i) for i in range(10)]
    source += [chr(i) for i in range(97, 97+26)]
    source += [chr(i) for i in range(65, 65+26)]
    alpha = ''.join(source) 
    new_label = []
    for index in label:
        new_label.appned(alpha[index])
    return new_label
    

In [ ]:
data = LoadTest('train/test/', transform)
cnn = ResNet18()

cnn.load_state_dict(torch.load('can.pt', map_location='cpu'))

cnn.eval()
pred = dict()

for i in range(len(data)):
    if (i+1)%1000 == 0:
        print('i = ', i+1)
    imgs, labels = data[i]
    imgs = torch.Tensor(imgs).reshape(1,1,30,150)
    temp = cnn(imgs)
    temp = output.view(-1, 62)
    temp = nn.functional.softmax(temp, dim=1)
    temp = torch.argmax(temp, dim=1)
    out = relabel(temp)
    pred[labels] = out

In [ ]:
labels = list()
for i in range(len(chu)):
    labels.append(''.join(chu[i]))

In [ ]:
np.savetxt('data.csv',labels, fmt='%s')